Ноутбук по проекту chicago_spark.  
МО

## Импорты

In [2]:
import os
import sys
import warnings
import gc

In [3]:
import pandas as pd
from scipy import stats
from scipy.signal import welch
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [4]:
from importlib import reload
import time
from tqdm import tqdm
from functools import reduce
from itertools import islice
from collections import defaultdict

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, HTML

In [6]:
from enviserv.dictan import DictAnalyzer # анализ словарей
import pandserv as pds # форматирование небольших пандас ДФ

In [7]:
from sparkserv import SparkApp, Cols
# в SparkApp упакованы функции создания спарк приложения 
# с определением IP мастер-ноды и с подключением к кластеру

# Col - класс для формирования коротких псевдонимов имен столбцов
# при этом исходные имена полей не меняются

In [8]:
import pyspark

from pyspark import SparkContext

from pyspark.sql import DataFrame as pydf
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, LongType
from pyspark import StorageLevel
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [9]:
# гео библиотеки
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point

## Создание сессии, загрузка данных

In [10]:
spark_app = SparkApp(my_logger_create_level = 'INFO')

INFO:spark_app:spark_master_ip: 172.18.0.2
INFO:spark_app:pyspark version: 3.4.1
INFO:spark_app:starting building spark app object: pyspark-taxi-forecasting
INFO:spark_app:Spark app object built as: <pyspark.sql.session.SparkSession object at 0x7fd4b7010f50>
INFO:spark_app:==================================================================
INFO:spark_app:Spark object can be accessed as the SparkApp_object.spark property
INFO:spark_app:==================================================================


In [11]:
spark_master_ip = spark_app.get_spark_master_ip()
# print(spark_master_ip)

INFO:spark_app:spark_master_ip: 172.18.0.2


In [12]:
# spark = spark_app.build_spark_app(spark_master_ip=spark_master_ip)
# spark = spark_app.spark

Для корректного завершения спарк-сессии (например, для переключения между ноутбуками) следует останавливать сессию полностью. Для этого использую метод .stop_spark_app() класса SparkApp

In [13]:
# spark_app.stop_spark_app()

In [14]:
print(spark_app.spark)

In [15]:
spark_app.build_spark_app()

INFO:spark_app:pyspark version: 3.4.1
INFO:spark_app:starting building spark app object: pyspark-taxi-forecasting
INFO:spark_app:Spark app object built as: <pyspark.sql.session.SparkSession object at 0x7fd4b7010f50>
INFO:spark_app:==================================================================
INFO:spark_app:Spark object can be accessed as the SparkApp_object.spark property
INFO:spark_app:==================================================================


Получим стандартный объект `spark` из созданного выше объекта `spark_app`

In [16]:
spark = spark_app.spark

Проверка работы спарк-объекта на кластере. Если все в порядке, то тест должен выполниться достаточно быстро и отобразить тестовый ДФ.  
```txt
+------------+-----------+
|student_name|student_age|
+------------+-----------+
|       Alice|         10|
|         Bob|         20|
+------------+-----------+
```

Если исходные образы кластера собраны с ошибкой, возможно "зависание" работы теста.

In [17]:
spark_app.test_spark_functionality()

Spark session created successfully.
DataFrame created successfully.
Alias DataFrame created successfully.
DataFrame data matches expected result.
+------------+-----------+
|student_name|student_age|
+------------+-----------+
|       Alice|         10|
|         Bob|         20|
+------------+-----------+

DataFrame show output matches expected output.

=======use======================
*      ____              __    *
*     / __/__  ___ _____/ /__  *
*    _\ \/ _ \/ _ `/ __/  '_/  *
*   /__ / .__/\_,_/_/ /_/\_\   *
*      /_/                     * 
        


In [18]:
da = DictAnalyzer()

In [19]:
# Функция простой рандомизированной выборки
def random_sample_dataframe(dataframe, percentage):
    # Генерируем случайные числа от 0 до 1 и фильтруем строки
    df = dataframe.filter(f.rand() < percentage)
    
    return df

In [20]:
# %%time
# agg_hour.coalesce(1).write.csv("/work/data/taxis_agg_hour_growth.csv", header=True, mode="overwrite")

### Загрузка данных

In [21]:
# %%time
# agg_hour = spark.read.load('/work/data/taxis_agg_hour_growth.csv', 
#                        format='csv', header='true'
#                         , inferSchema='true'
#                        )

In [22]:
# agg_hour.count()

In [23]:
# agg_hour.cache()

In [24]:
# agg_null = agg_hour.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in agg_hour.columns]).toPandas()

In [25]:
# agg_null = agg_null.transpose()

In [26]:
# agg_null[agg_null.iloc[:, 0] > 0]

Загрузим данные по исключаемым (высокоррелированным) полям

In [27]:
excl_f = pd.read_csv('/work/data/excluded_fields.csv')

In [28]:
# Функция для преобразования DataFrame в словарь
def get_excuded_fields_to_dict(df):
    excluded_fields_tot = {}
    grouped = df.groupby(['ct', 'ca'])
    for (ct, ca), group in grouped:
        excluded_fields_tot[(ct, ca)] = group['excluded_field'].tolist()
    return excluded_fields_tot

In [29]:
excluded_fields_tot = get_excuded_fields_to_dict(excl_f)

In [30]:
da.print_dict(dict(islice(excluded_fields_tot.items(), 3)))

{
    (10000000000, 91): ['time_p', 'miles_p', 'farem_p', 'comp1_p', 'comp3_p', 'comp4_p', 'comp5_p', 'extrasm_p']
    (12000000001, 53): ['time_p', 'farem_p', 'trips_d', 'time_d', 'miles_d', 'farem_d', 'totalm_p', 'velocity_p_growth_3_to_2']
    (12000000002, 75): ['time_p', 'miles_p', 'farem_p', 'tipsm_p', 'comp5_p', 'trips_d', 'time_d', 'miles_d', 'farem_d', 'tipsm_d', 'comp4_p']
}


Будем считать, что от мультиколлинеарности в линейных моделях с помощью исключения этих полей получится избавиться.

In [31]:
# %%time
# agg_hour.coalesce(1).write.csv("/work/data/taxis_agg_hour_result.csv", header=True, mode="overwrite")

In [32]:
%%time
data = spark.read.load('/work/data/taxis_agg_hour_result.csv', 
                       format='csv', header='true'
                        , inferSchema='true'
                       )

CPU times: user 0 ns, sys: 20.6 ms, total: 20.6 ms
Wall time: 49.3 s


In [33]:
print(pds.gvf(data.count()))

3'062'043


In [34]:
data.cache()

DataFrame[ct: bigint, ca: int, hour_start: timestamp, time_p: int, miles_p: double, velocity_p: double, farem_p: double, tipsm_p: double, tollsm_p: double, extrasm_p: double, totalm_p: double, comp1_p: int, comp2_p: int, comp3_p: int, comp4_p: int, comp5_p: int, compless5_p: int, trips_d: int, time_d: int, miles_d: double, velocity_d: double, farem_d: double, tipsm_d: double, tollsm_d: double, extrasm_d: double, totalm_d: double, comp1_d: int, comp2_d: int, comp3_d: int, comp4_d: int, comp5_d: int, compless5_d: int, cumulative_balance: int, trips_p_growth_1_to_0: double, trips_p_growth_2_to_1: double, trips_p_growth_3_to_2: double, trips_p_growth_4_to_3: double, trips_d_growth_1_to_0: double, trips_d_growth_2_to_1: double, trips_d_growth_3_to_2: double, trips_d_growth_4_to_3: double, velocity_p_growth_1_to_0: double, velocity_p_growth_2_to_1: double, velocity_p_growth_3_to_2: double, velocity_p_growth_4_to_3: double, velocity_d_growth_1_to_0: double, velocity_d_growth_2_to_1: double, v

### Фрагмент данных

Отберем несколько полей и один район для проверки

In [35]:
f_to_sel = [
    'ct', 'ca', 'hour_start', 'trips_target', 'time_p', 'miles_p', 'velocity_p',
]

data_sample = data.select(*f_to_sel).filter(f.col('ct')==17031090200)
data_sample.show(3)
# data_sample.cache()

+-----------+---+-------------------+------------+------+-------+--------------------+
|         ct| ca|         hour_start|trips_target|time_p|miles_p|          velocity_p|
+-----------+---+-------------------+------------+------+-------+--------------------+
|17031090200|  6|2021-01-08 01:00:00|           1|     0|    0.0|                 0.0|
|17031090200|  6|2021-01-08 02:00:00|           2|   706|   3.24|0.004589235127478754|
|17031090200|  6|2021-01-08 03:00:00|           1|   990|    2.9|0.002929292929292929|
+-----------+---+-------------------+------------+------+-------+--------------------+
only showing top 3 rows



In [36]:
" ".join(excluded_fields_tot[(17031090200,6)])

'time_p miles_p farem_p tipsm_p comp1_p comp3_p comp4_p trips_d time_d miles_d farem_d tipsm_d extrasm_d comp2_d comp3_d comp4_d extrasm_p comp5_p'

Данные загрузились. Общее количество и отображение фрагмента соответствуют ожиданиям. Исключаемые поля в доступе.

### Инвентаризация объектов

Датафреймы pyspark, если они были зарегестрированы в качестве представлений (таблиц), можно молучить используя catalog. Но если это просто объекты Питон типа pyspark.sql.dataframe.DataFrame, такие дф надо искать среди всех объектов в глобальной области видимости программы. Воспользуемся написанной функцией. При этом возможна путаница, связанная с "магической" служебной переменной Юпитера с именем "__" (последнее сохраненное значение ячейки). Такое имя может заменить реальное имя объекта. Решается двухкратным вызовом, например, `type(__)`.

In [37]:
# 
type(__)

pyspark.sql.dataframe.DataFrame

In [38]:
type(__)

str

In [39]:
# global_namespace=globals()

In [40]:
# reload(pds)

In [41]:
# ob = pds.inventory_objects(globals())
# ob_df = ob[ob['Type'].str.contains('DataFrame')]
# ob_df = pds.get_df_formated(ob_df,"'",0,40)
# print(ob_df)

In [42]:
def get_spark_df_list(globe_dict):
    objects = pds.inventory_objects(globe_dict)
    spark_df = objects[objects['Type'].str.contains('Spark DataFrame') & 
                       ~objects['Name'].str.startswith('DataFrame[') & 
                       ~objects['Name'].str.startswith('_')].copy()
    del objects
    spark_df['cached'] = spark_df.apply(lambda x: eval(x['Name']).is_cached \
            if not (x['Name'].startswith("DataFrame[") or x['Name'].startswith("_")) else None, axis=1)
    spark_df['cache_level'] = spark_df.apply(lambda x: eval(x['Name']).storageLevel \
            if not (x['Name'].startswith("DataFrame[") or x['Name'].startswith("_")) else None, axis=1)
    return spark_df

In [43]:
# spark_df = get_spark_df_list(globals())

In [44]:
# spark_df

## Порядок действий МО

In [45]:
%%time
print(pds.gvf(data.count()))

3'062'043
CPU times: user 14.7 ms, sys: 1.88 ms, total: 16.6 ms
Wall time: 24.3 s


Общее описание порядка действий

Некоторая информация связанная с векторизацией признаков.  
Сохранить датафрейм со столбцом вектором возможно, например, в формате parquet; csv не поддерживает вектор-столбцы. Загруженный паркет-файл вызывает ошибку при выполнении действий.   
При кэшировании векторизированного дф (с использованием уровня хранения Disk Memory Deserialized 1x Replicated - уровень, который присваивается по умолчанию при вызове .cache()), такой кэшированный датафрейм с вектор-столбцом не получается обрабатывать из-за нехватки памяти.  

Пока остановился на варианте:  
- 1. делю входной дф на трэйн и тест, кэширую их .cache() (Disk Memory)
- 2. получаю фрагмент geo_df_base_train/test для гео-ключа из трэйн дф, кэширую в ОЗУ geo_df_base_train/test
- 3. векторизирую фичи geo_df_base_train/test,
- 4. создаю рабочий дф geo_df_train/test дф с нужными полями (гео-ключ, время, метка, фича-вектор)
- 5. кэширую в ОЗУ geo_df_train/test;
- 6. создаю для geo_df_train скалер
- 7. масштабирую-обновляю geo_df_train/test, кэширую в ОЗУ,
- 8. обучаю модель лин.регрессии на трэйне (включая тюнинг модели)
- 9. сохраняю в пандас дф для данного гео-ключа полученный объект-модель и метрики на трэйне
- 10. получаю предикт на трэйне и тесте и добавляю столбец предикта в исходные train/test по гео-ключу, обновляю их и кэширую .cache()

### 1. делю входной дф на трэйн и тест, кэширую их .cache() (Disk Memory)

In [46]:
# Разделение данных на тренировочный и тестовый наборы
train = data.filter(f.year(f.col("hour_start")) < 2024)
test = data.filter(f.year(f.col("hour_start")) == 2024)

In [47]:
train.cache()

DataFrame[ct: bigint, ca: int, hour_start: timestamp, time_p: int, miles_p: double, velocity_p: double, farem_p: double, tipsm_p: double, tollsm_p: double, extrasm_p: double, totalm_p: double, comp1_p: int, comp2_p: int, comp3_p: int, comp4_p: int, comp5_p: int, compless5_p: int, trips_d: int, time_d: int, miles_d: double, velocity_d: double, farem_d: double, tipsm_d: double, tollsm_d: double, extrasm_d: double, totalm_d: double, comp1_d: int, comp2_d: int, comp3_d: int, comp4_d: int, comp5_d: int, compless5_d: int, cumulative_balance: int, trips_p_growth_1_to_0: double, trips_p_growth_2_to_1: double, trips_p_growth_3_to_2: double, trips_p_growth_4_to_3: double, trips_d_growth_1_to_0: double, trips_d_growth_2_to_1: double, trips_d_growth_3_to_2: double, trips_d_growth_4_to_3: double, velocity_p_growth_1_to_0: double, velocity_p_growth_2_to_1: double, velocity_p_growth_3_to_2: double, velocity_p_growth_4_to_3: double, velocity_d_growth_1_to_0: double, velocity_d_growth_2_to_1: double, v

In [48]:
test.cache()

DataFrame[ct: bigint, ca: int, hour_start: timestamp, time_p: int, miles_p: double, velocity_p: double, farem_p: double, tipsm_p: double, tollsm_p: double, extrasm_p: double, totalm_p: double, comp1_p: int, comp2_p: int, comp3_p: int, comp4_p: int, comp5_p: int, compless5_p: int, trips_d: int, time_d: int, miles_d: double, velocity_d: double, farem_d: double, tipsm_d: double, tollsm_d: double, extrasm_d: double, totalm_d: double, comp1_d: int, comp2_d: int, comp3_d: int, comp4_d: int, comp5_d: int, compless5_d: int, cumulative_balance: int, trips_p_growth_1_to_0: double, trips_p_growth_2_to_1: double, trips_p_growth_3_to_2: double, trips_p_growth_4_to_3: double, trips_d_growth_1_to_0: double, trips_d_growth_2_to_1: double, trips_d_growth_3_to_2: double, trips_d_growth_4_to_3: double, velocity_p_growth_1_to_0: double, velocity_p_growth_2_to_1: double, velocity_p_growth_3_to_2: double, velocity_p_growth_4_to_3: double, velocity_d_growth_1_to_0: double, velocity_d_growth_2_to_1: double, v

In [49]:
# %%time
# print(pds.gvf(train.count()))

In [50]:
# %%time
# print(pds.gvf(test.count()))

In [51]:
target_variable_name = 'trips_target'
# исключаемые из расчетов ключевые поля для всех районов
key_cols = ['ct', 'ca', 'hour_start']
# поля которые будут в рабочих ДФ
# ['ct', 'ca', 'hour_start', 'trips_target', 'features']
# 'features' - немасштабированный вектор-столбец
selectedCols = key_cols + [target_variable_name, 'features']

In [52]:
# Получение уникальных комбинаций районов и округов
geo_keys = data.select('ct', 'ca').distinct().collect()

### 2. получаю фрагмент geo_df_base_train/test для гео-ключа из трэйн дф, кэширую в ОЗУ geo_df_base_train/test

In [53]:
# получаю фрагмент geo_df_base_train/test
# для гео-ключа из трэйн/тест дф
def get_geo_df_base(point, train, test):
    ct = point[0]
    ca = point[1]
    # Фильтрация данных для текущего района и округа
    geo_data_base_train = train.filter((f.col('ct') == ct) & (f.col('ca') == ca))
    geo_data_base_test = test.filter((f.col('ct') == ct) & (f.col('ca') == ca))

    return geo_data_base_train, geo_data_base_test

In [54]:
def cache_df_in_ram(df):
    df.persist(StorageLevel.MEMORY_ONLY)

In [55]:
# point = (17031090200,6)

In [56]:
# geo_data_base_train, geo_data_base_test = get_geo_df_base(point, train, test)

In [57]:
# # кэширую в ОЗУ geo_df_base_train/test
# cache_df_in_ram(geo_data_base_train)
# cache_df_in_ram(geo_data_base_test)

### 3. векторизирую фичи geo_df_base_train/test

In [58]:
# векторизация указанных полей датафрейма
def assemble_vectors(df, features_list, selectedCols):
    stages = []
    assembler = VectorAssembler(inputCols=features_list, outputCol='features')
    stages = [assembler]
    pipeline = Pipeline(stages=stages)
    # assemble model
    assembleModel = pipeline.fit(df)
    # apply assembler model on data
    df = assembleModel.transform(df).select(selectedCols)
    return df

In [59]:
def get_features_list(point,df,target_variable_name):
    base_columns = df.columns
    all_excluded_columns = key_cols + [target_variable_name] + excluded_fields_tot[point]
    features_list = [col for col in base_columns if col not in all_excluded_columns]
    return features_list

In [60]:
# features_list = get_features_list(point,data,target_variable_name)

In [61]:
# " ".join(features_list)

### 4. создаю рабочий дф geo_df_train/test дф с нужными полями (гео-ключ, время, метка, фича-вектор)

In [62]:
# geo_df_train = assemble_vectors(geo_data_base_train,features_list,selectedCols)
# geo_df_test = assemble_vectors(geo_data_base_test,features_list,selectedCols)

### 5. кэширую в ОЗУ geo_df_train/test

In [63]:
# print(pds.gvf(geo_df_train.count()))

In [64]:
# print(pds.gvf(geo_df_test.count()))

In [65]:
# # кэширую в ОЗУ geo_df_base_train/test
# cache_df_in_ram(geo_df_train)
# cache_df_in_ram(geo_df_test)

### 6. создаю для geo_df_train скалер

In [66]:
def get_fitted_scaler(df, featureCol):
    outputCol = featureCol+"_scaled"
    stages = []
    scaler = StandardScaler(inputCol = featureCol,
                            outputCol=outputCol,
                            withStd=True, withMean=True)
    stages = [scaler]
    pipeline = Pipeline(stages=stages)
    scaledAssembleModel = pipeline.fit(df)

    return scaledAssembleModel

In [67]:
# geo_df_train.columns

In [68]:
# geo_df_train_scaler = get_fitted_scaler(geo_df_train, 'features')

### 7. масштабирую-обновляю geo_df_train/test, кэширую в ОЗУ

In [69]:
# geo_df_train = geo_df_train_scaler.transform(geo_df_train)

In [70]:
# geo_df_test = geo_df_train_scaler.transform(geo_df_test)

In [71]:
# print(pds.gvf(geo_df_train.count()))

In [72]:
# print(pds.gvf(geo_df_test.count()))

In [73]:
# geo_df_train.show(3)

### 8. обучаю модель лин.регрессии на трэйне

In [110]:
def lin_regr_fit(point,
                 scaled_geo_train,
                 scaled_geo_test,
                 features_col = 'features',
                 scaled_features_col = 'features_scaled',
                 target_variable_name = 'trips_target',
                 exc_cols = None
):
    if exc_cols is None:
        exc_cols = ['ct', 'ca', 'hour_start']
    
    reg = LinearRegression(
        featuresCol = scaled_features_col,
        labelCol = target_variable_name
    )
    
    ct = point[0]
    ca = point[1]
        
    selectedCols = exc_cols + [target_variable_name] + [scaled_features_col]

    reg_model = reg.fit(scaled_geo_train)

    geo_train_transformed = reg_model.transform(scaled_geo_train)
    geo_test_transformed = reg_model.transform(scaled_geo_test)
    
    # Создание объекта RegressionEvaluator для оценки модели
    evaluator = RegressionEvaluator()

    # Настройка параметров оценки (RMSE и MAPE)
    evaluator.setMetricName("rmse")
    evaluator.setPredictionCol("prediction")
    evaluator.setLabelCol(target_variable_name)

    # Оценка модели
    rmse_train = evaluator.evaluate(geo_train_transformed)
    rmse_test = evaluator.evaluate(geo_test_transformed)
    
    # Аналогично для MAE
    evaluator.setMetricName("mae")
    mae_train = evaluator.evaluate(geo_train_transformed)
    mae_test = evaluator.evaluate(geo_test_transformed)
    
    # mae_train, mae_test, mape_train, mape_test = 0,0,0,0
    # print("mae_train:",mae_train,"mae_test",mae_test,"mape_train:",mape_train,"mape_test",mape_test)
    
    # # Вычисление MAE вручную для train
    # geo_train_transformed = geo_train_transformed.withColumn("abs_error", 
    #     f.abs(geo_train_transformed["prediction"] - geo_train_transformed[target_variable_name])
    # )
    # mae_train = geo_train_transformed.selectExpr("mean(abs_error) as mae").collect()[0]["mae"]
    
    # # Вычисление MAE вручную для test
    # geo_test_transformed = geo_test_transformed.withColumn("abs_error", 
    #     f.abs(geo_test_transformed["prediction"] - geo_test_transformed[target_variable_name])
    # )
    # mae_test = geo_test_transformed.selectExpr("mean(abs_error) as mae").collect()[0]["mae"]

    # print("mae_train:",mae_train,"mae_test",mae_test)

    # Получение предсказаний и реальных значений
    # Вычисление MAPE
    geo_train_transformed = geo_train_transformed.withColumn("abs_pct_error", 
        f.abs((geo_train_transformed["prediction"] - geo_train_transformed[target_variable_name]) \
            / geo_train_transformed[target_variable_name])
    )
    mape_train = geo_train_transformed.selectExpr("mean(abs_pct_error) * 100 as mape")\
        .collect()[0]["mape"]
    geo_test_transformed = geo_test_transformed.withColumn("abs_pct_error", 
        f.abs((geo_test_transformed["prediction"] - geo_test_transformed[target_variable_name]) \
            / geo_test_transformed[target_variable_name])
    )
    mape_test = geo_test_transformed.selectExpr("mean(abs_pct_error) * 100 as mape")\
        .collect()[0]["mape"]
    # print("mape_train:",mape_train,"mape_test",mape_test)
    
    return reg_model, geo_train_transformed, geo_test_transformed, \
            rmse_train, mae_train, mape_train, \
            rmse_test, mae_test, mape_test

In [75]:
# ванильный тюнинг не используется - не меняет результатов
# def lin_regr_cv_fit(point,
#                  scaled_geo_train,
#                  scaled_geo_test,
#                  features_col='features',
#                  scaled_features_col='features_scaled',
#                  target_variable_name='trips_target',
#                  exc_cols=None,
#                  max_iter=100, 
#                  reg_param=0.0, 
#                  elastic_net_param=0.0, 
#                  param_grid=None,
#                  num_folds=3):
#     if exc_cols is None:
#         exc_cols = ['ct', 'ca', 'hour_start']
    
#     # Создание модели линейной регрессии с параметрами
#     reg = LinearRegression(
#         featuresCol=scaled_features_col,
#         labelCol=target_variable_name,
#         maxIter=max_iter,
#         regParam=reg_param,
#         elasticNetParam=elastic_net_param
#     )
    
#     ct = point[0]
#     ca = point[1]
    
#     selectedCols = exc_cols + [target_variable_name] + [scaled_features_col]

#     # Настройка параметров для кросс-валидации
#     if param_grid is None:
#         param_grid = (ParamGridBuilder()
#                       .addGrid(reg.regParam, [0.0, 0.1, 0.01])
#                       .addGrid(reg.elasticNetParam, [0.0, 0.5, 1.0])
#                       .build())
    
#     # Создание кросс-валидатора
#     evaluator = RegressionEvaluator(metricName="rmse", predictionCol="prediction", labelCol=target_variable_name)
#     crossval = CrossValidator(estimator=reg,
#                               estimatorParamMaps=param_grid,
#                               evaluator=evaluator,
#                               numFolds=num_folds)
    
#     # Обучение модели с кросс-валидацией
#     cv_model = crossval.fit(scaled_geo_train)
#     reg_model = cv_model.bestModel
    
#     geo_train_transformed = reg_model.transform(scaled_geo_train)
#     geo_test_transformed = reg_model.transform(scaled_geo_test)
    
#     # Оценка модели
#     rmse_train = evaluator.evaluate(geo_train_transformed)
#     rmse_test = evaluator.evaluate(geo_test_transformed)
    
#     evaluator.setMetricName("mae")
#     mae_train = evaluator.evaluate(geo_train_transformed)
#     mae_test = evaluator.evaluate(geo_test_transformed)

#     
    
#     # Вычисление MAPE
#     geo_train_transformed = geo_train_transformed.withColumn("abs_pct_error", 
#         f.abs((geo_train_transformed["prediction"] - geo_train_transformed[target_variable_name]) \
#             / geo_train_transformed[target_variable_name])
#     )
#     mape_train = geo_train_transformed.selectExpr("mean(abs_pct_error) * 100 as mape")\
#         .collect()[0]["mape"]
#     geo_test_transformed = geo_test_transformed.withColumn("abs_pct_error", 
#         f.abs((geo_test_transformed["prediction"] - geo_test_transformed[target_variable_name]) \
#             / geo_test_transformed[target_variable_name])
#     )
#     mape_test = geo_test_transformed.selectExpr("mean(abs_pct_error) * 100 as mape")\
#         .collect()[0]["mape"]
#     
    
#     return reg_model, geo_train_transformed, geo_test_transformed, \
#             rmse_train, mae_train, mape_train, \
#             rmse_test, mae_test, mape_test

In [76]:
# rmse, mae, mape = 0, 0, 0

In [77]:
# reg_model, geo_train_transformed, geo_test_transformed, rmse, mae, mape =\
# lin_regr_fit(point,
#                  geo_df_train,
#                  geo_df_test,
#                  features_col = 'features',
#                  scaled_features_col = 'features_scaled',
#                  target_variable_name = 'trips_target',
#                  exc_cols = None
# )

In [78]:
# geo_train_transformed.show(3)

In [79]:
# rmse, mae, mape

In [80]:
# reg_model.coefficients

### 9. сохраняю в пандас дф для данного гео-ключа полученный объект-модель и метрики на трэйне

In [81]:
# models = pd.DataFrame(columns=["ct","ca", "reg_model", "rmse", "mae", "mape","features","coefficients","intercept"])

In [82]:
def add_model_metrics(df=None, point=None, reg_model=None,
                      rmse_train=None, mae_train=None, mape_train=None,
                      rmse_test=None, mae_test=None, mape_test=None,
                     features_list=None,coefficients=None,intercept=None):
    # Создание нового DataFrame, если он не был передан
    if df is None:
        df = pd.DataFrame(columns=["ct","ca", "reg_model",
                                   "rmse_train", "mae_train", "mape_train",
                                   "rmse_test", "mae_test", "mape_test",
                                   "features","coefficients","intercept"])
    
    # Создание новой строки с метриками
    new_row = {
        "ct": point[0],
        "ca": point[1],
        "reg_model": reg_model,
        "rmse_train": rmse_train,
        "mae_train": mae_train,
        "mape_train": mape_train,
        "rmse_test": rmse_test,
        "mae_test": mae_test,
        "mape_test": mape_test,
        "features": features_list,
        "coefficients": coefficients,
        "intercept": intercept
    }
    
    # Добавление новой строки в DataFrame
    new_df = pd.DataFrame([new_row])
    updated_df = pd.concat([df, new_df], ignore_index=True)
    
    return updated_df

In [83]:
# models = add_model_metrics(df=models,
#                            point=point,
#                            reg_model=reg_model,
#                            rmse=rmse, mae=mae, mape=mae,
#                            features_list=features_list,
#                            coefficients=reg_model.coefficients,
#                            intercept=reg_model.intercept
#                           )

In [84]:
# models

### 10. получаю предикт на трэйне и тесте и добавляю столбец предикта в исходные train/test по гео-ключу, обновляю их и кэширую .cache()

In [85]:
def add_predictions_to_base_data(geo_data_base_train,
                                 geo_data_base_test,
                                 geo_train_transformed, 
                                 eo_test_transformed,
                                 join_columns):
    # Объединение данных с предсказаниями и исходных данных по указанным столбцам
    geo_data_base_train_with_predictions = geo_data_base_train.join(
        geo_train_transformed.select(*join_columns, "prediction"),
        on=join_columns,
        how="left"
    )
    
    geo_data_base_test_with_predictions = geo_data_base_test.join(
        geo_test_transformed.select(*join_columns, "prediction"),
        on=join_columns,
        how="left"
    )
    return geo_data_base_train_with_predictions,\
           geo_data_base_test_with_predictions

In [86]:
# geo_data_base_train, geo_data_base_test = \
# add_predictions_to_base_data(geo_data_base_train,
#                              geo_data_base_test,
#                              geo_train_transformed, 
#                              geo_test_transformed,
#                              key_cols)

In [87]:
# f_to_sel = key_cols +[target_variable_name]+['prediction']

In [88]:
# geo_data_base_train.select(*f_to_sel).show(10)

### Все шаги


In [118]:
points_list = geo_keys[:]
# points_list

In [119]:

models = pd.DataFrame(columns=["ct","ca", "reg_model",
                               "rmse_train", "mae_train", "mape_train",
                               "rmse_test", "mae_test", "mape_test",
                               "features","coefficients","intercept"])

train_res = None
test_res = None

for point in tqdm(points_list):
    # 2. получаю фрагмент geo_df_base_train/test
    # для гео-ключа из трэйн дф,
    geo_data_base_train, geo_data_base_test = get_geo_df_base(point, train, test)
    # кэширую в ОЗУ geo_df_base_train/test
    cache_df_in_ram(geo_data_base_train)
    cache_df_in_ram(geo_data_base_test)
    # 3. векторизирую фичи geo_df_base_train/test
    features_list = get_features_list(point,train,target_variable_name)
    # 4. создаю рабочий дф geo_df_train/test дф
    # с нужными полями (гео-ключ, время, метка, фича-вектор)
    geo_df_train = assemble_vectors(geo_data_base_train,features_list,selectedCols)
    geo_df_test = assemble_vectors(geo_data_base_test,features_list,selectedCols)
    # 5. кэширую в ОЗУ geo_df_base_train/test
    cache_df_in_ram(geo_df_train)
    cache_df_in_ram(geo_df_test)
    # 6. создаю для geo_df_train скалер
    geo_df_train_scaler = get_fitted_scaler(geo_df_train, 'features')
    # 7. масштабирую-обновляю geo_df_train/test, кэширую в ОЗУ
    geo_df_train = geo_df_train_scaler.transform(geo_df_train)
    geo_df_test = geo_df_train_scaler.transform(geo_df_test)
    cache_df_in_ram(geo_df_train)
    cache_df_in_ram(geo_df_test)
    # 8. обучаю модель лин.регрессии на трэйне
    # получаю модель прогнозы и метрики
    reg_model, geo_train_transformed, geo_test_transformed, \
            rmse_train, mae_train, mape_train, \
            rmse_test, mae_test, mape_test =\
    lin_regr_fit(point,
                 geo_df_train,
                 geo_df_test,
                 features_col = 'features',
                 scaled_features_col = 'features_scaled',
                 target_variable_name = 'trips_target',
                 exc_cols = None
    )
    # 9. сохраняю в пандас дф для данного
    # гео-ключа полученный объект-модель и метрики на трэйне
    models = add_model_metrics(df=models,
                           point=point,
                           reg_model=reg_model,
                           rmse_train=rmse_train, mae_train=mae_train, mape_train=mape_train,
                           rmse_test=rmse_test, mae_test=mae_test, mape_test=mape_test,
                           features_list=features_list,
                           coefficients=reg_model.coefficients,
                           intercept=reg_model.intercept
                          )
    # 10. получаю предикт на трэйне и тесте и добавляю столбец
    # предикта в исходные train/test по гео-ключу,
    # обновляю их и кэширую .cache()
    geo_data_base_train, geo_data_base_test = \
    add_predictions_to_base_data(geo_data_base_train,
                             geo_data_base_test,
                             geo_train_transformed, 
                             geo_test_transformed,
                             key_cols)
    # 11. Добавляю полученный датафрейм в результат
    if train_res is None:
        train_res = geo_data_base_train
    else:
        train_res = train_res.union(geo_data_base_train)
    # train_res.cache()
    if test_res is None:
        test_res = geo_data_base_test
    else:
        test_res = test_res.union(geo_data_base_test)
    # test_res.cache()
    # return train_res, test_res, models

100%|██████████| 103/103 [22:34<00:00, 13.15s/it]


In [100]:
# train_res, test_res, models = all_steps_taxi(points_list)

In [91]:
# models.head()

In [120]:
models[["rmse_train", "mae_train", "mape_train",
        "rmse_test", "mae_test", "mape_test",]].head(3)

,rmse_train,mae_train,mape_train,rmse_test,mae_test,mape_test
0,3.361335,2.549306,49.572514,3.414791,2.568554,47.209217
1,8.174596,6.047635,34.219294,9.080190,6.607081,33.351287
2,3.070174,2.353955,42.468547,3.186198,2.422917,44.098728


In [121]:
models[["rmse_train", "mae_train", "mape_train",
        "rmse_test", "mae_test", "mape_test",]].describe()

,rmse_train,mae_train,mape_train,rmse_test,mae_test,mape_test
count,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000
mean,2.810005,1.945298,55.284813,3.140904,2.153357,55.831406
std,3.234132,2.232060,12.121400,4.106706,2.800895,12.498734
min,0.322773,0.170916,23.589202,0.348340,0.193614,17.662376
25%,0.907780,0.636100,46.940058,0.970418,0.624346,47.566825
50%,1.457101,1.050917,52.464250,1.600986,1.142249,52.625650
75%,3.560571,2.401178,61.051702,3.410993,2.246624,63.417640
max,17.673250,11.626479,88.378262,22.965733,16.667246,89.031368


## Результат

**Задача проекта:** Разработка модели машинного обучения для прогнозирования спроса на такси на основе исторических данных о поездках.

**Основные этапы проекта:**

1. **Загрузка и сохранение данных:** Данные за 2021-2024/2 гг - около 20М строк. Три csv файла объединены в один датафрейм. Поскольку сложные планы выполнения действий с увеличением операций значительно увеличивают время преобразований, каждый логический этап завершался сохранением файла.
01_eda_geo_keys_all.ipynb

2. **Предварительный анализ данных:** Анализ в основном заключался в поиске возможных способов восполнить пропуски в гео-ключах (код района, округа, координаты). Использовался внешний датафрейм также с сайта ситиофЧикаго, использовался метод ближайших соседей. Очень много джойнов и проверок внутри датафрейма. Также удбраны выбросы (поездки дороже 315 баксов, более 7900 секунд и более 53 миль). Посмотрел 3Д гистограмму частот. О'Хара и центр делают кассу.
01_eda_geo_keys_all.ipynb

3.  **Подготовка данных:** Были выполнены следующие шаги по подготовке данных:
   - Группировка по 15 минутным интервалам, выравниевание сетки интервалов
   - Доп группировка по гео-ключам. Оставил отдельно крупные районы в некоторых округах. Но основная часть поездок привязана в большинстве округов к одному району в округе. В результате получил не 77(78) гео-ключей, а 103.
   - Расчет средней скорости за 15мин интервал, остальные показатели суммируются
   - Группировка по часовым интервалам. 15минутные скорости и Трипсы внутри часа выстроены в 4 столбца и посчитан внутричасовые приросты для 15минутных Трипсов и скоростей
   - Для каждого района вычислены высококоррелированные поля (без Трипса - таргета), списки одной стороны из таких пар полей сохранены в отдельный ДФ. Так при моделировании будем исключать эти поля для избежания мультиколлинеарности в линейной регрессии.
   - Вычислены наиболее значимые частоты. Посчитанв лаги, скользящие средние и приросты скользящих для лагов. Это все суть новые фичи.
   - Разделение данных на тренировочную (2021-2023гг), тестовую (2024) выборки.
   - Категориальных признаков не было.
   - сдвинул назад Трипс-таргет, чтобы он стал таргетом.
02_aggfe.ipynb

4. **Машинное обучение:** Использовалась простая линейная регрессия без тюнинга и без регуляризации. Считались модели для каждого гео-ключа. Вырезалась и кэшировалась в ОЗУ (для скорости) часть датафрейма, на которой обучалась модель. Данные моделей собирались в итоговый пандас датафрейм. Из вырезанного ДФ удалялись мультколлинеарные поля. Фичи векторизировались и масштабировались (скалер обучен на трэйне и применен на тесте).

5. **Оценка модели:** Валидационная выборка не выделялась и кросс-валидация не проводилась. Посчитаны метрики RMSE, MAE, MAPE. Средние значения соответственно:
```yaml
| Statistic | RMSE_Train | MAE_Train | MAPE_Train | RMSE_Test | MAE_Test | MAPE_Test |
|-----------|------------|-----------|------------|-----------|----------|-----------|
| Mean      | 2.810005   | 1.945298  | 55.284813   | 3.140904  | 2.153357 | 55.831406 |
| Std       | 3.234132   | 2.232060  | 12.121400   | 4.106706  | 2.800895 | 12.498734 |
| Min       | 0.322773   | 0.170916  | 23.589202   | 0.348340  | 0.193614 | 17.662376 |
| 25%       | 0.907780   | 0.636100  | 46.940058   | 0.970418  | 0.624346 | 47.566825 |
| 50%       | 1.457101   | 1.050917  | 52.464250   | 1.600986  | 1.142249 | 52.625650 |
| 75%       | 3.560571   | 2.401178  | 61.051702   | 3.410993  | 2.246624 | 63.417640 |
| Max       | 17.673250  | 11.626479 | 88.378262   | 22.965733 | 16.667246 | 89.031368 |

```
Модель имеет хорошую обобщающую способность.  
Важность признаков не анализировалась.

**Результаты проекта:**

- LinearRegression справляется.
- Модель демонстрирует хорошие средние показатели качества с MAE меньше 2.

**Выводы:**

В процессе работы над проектом выучил ООП, немного бэкенда, Докер, немного Баша, логирование, ну и конечно же PySpark. Создан вычислительный кластер. Проведена большая работа по предобработке данных и конструированию признаков. Как всегда предподготовка обеспечивает успех даже для простой модели.

**Дальнейшая работа над проектом "Прогнозирование спроса на такси" может включать в себя следующие улучшения:**

1. **Инженерия признаков:**

- Разработать дополнительные временные признаки, такие как выходные дни, праздники, сезонные факторы.
- Исследовать и использовать информацию о погодных условиях, событиях в городе, которые могут влиять на спрос на такси.

2. **Улучшение модели:**

- Попробовать другие модели машинного обучения, такие как нейронные сети, временные ряды ARIMA или SARIMA, для сравнения и выбора наилучшей.
- Дополнительная настройка гиперпараметров модели для улучшения ее производительности.
- Рассмотреть ансамблевые методы, чтобы комбинировать преимущества разных моделей.
- прогнозировать именно всплески, т.е. превышение прогнозов (используя дискретизацию отклонений от прогноза и классификацию таких отклонений - как вариант)
- Графический анализ - пики, впадины..

3. **Связь между районами:**

- Оедельно улучшить (проанализировать) результаты по районам с плохими метриками
- Оценить потоки между районами
- Классифицировать ряды. Анализировать ряды не изолированно по районам.

## Project FIN